In [ ]:
!pip install -q langchain==0.1.16 langchain-openai==0.0.8 langchain-community==0.0.32 langchain_experimental tabulate

In [ ]:
!pip install --upgrade numpy pandas -q

In [4]:
import pandas as pd
import os

from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents import AgentType

In [5]:
csv_file_path = 'sales_report.csv'

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_base = "https://openrouter.ai/api/v1", openai_api_key = "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", model = "google/gemma-3-27b-it:free",temperature=0.5) #max_tokens=100

In [ ]:
print("\nCreating CSV Agent...")
agent_executor = create_csv_agent(
    llm,
    csv_file_path,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True, # Allow the agent to execute python code
    handle_parsing_errors=True

)
print("Agent is ready.")

In [ ]:
AGENT_PREFIX = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you.
You have access to a single tool: `python_repl_ast`.
Use this to process the dataframe and return the final answer.

Your response MUST be in the following format:

Question: the user's question
Thought: I need to use pandas to find the answer. I will write the code to do that.
Action: python_repl_ast
Action Input: a single, valid line of pandas code to run

Example:
Question: how many rows are there?
Thought: I need to count the rows in the dataframe.
Action: python_repl_ast
Action Input: df.shape[0]
"""

print("\nCreating CSV Agent with a guiding prefix...")
agent_executor = create_csv_agent(
    llm,
    csv_file_path,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    prefix=AGENT_PREFIX
)
print("Agent is ready.")

In [ ]:
query1 = "What was the total sale amount for the North region?"
print(f"\nQuestion: {query1}")
response1 = agent_executor.invoke(query1)
print("Answer:", response1['output'])


In [ ]:
query1 = "how many laptops sold in south region"
print(f"\nQuestion: {query1}")
response1 = agent_executor.invoke(query1)
print("Answer:", response1['output'])